In [1]:
#Libraries
import os
import pandas as pd
import numpy as np
import keras
import tensorflow
import sklearn
from matplotlib import pyplot as plt
import sys
import surfboard
from surfboard.feature_extraction_multiprocessing import extract_features

Using TensorFlow backend.


In [2]:
#Misc. adjustments
pd.set_option('max_colwidth', 200)

In [3]:
#Listing files
filelist = os.listdir("Source/vox")

#Read files into pandas
source = pd.DataFrame(filelist)
#Rename file name column
source = source.rename(columns={0:'file'})
#Filter out DS files from macOS
source = source[source.file != ".DS_Store"]
#Reset index
source = source.reset_index(drop = True)

#Extraction of variables from filenames
speaker_id = []
condition = []
index = []

for i in range (0, len(source)):
    id = source["file"][i][0:2]
    con = source["file"][i][3]
    indx = source["file"][i][5:7]
    
    if id[0:1] == "0":
        speaker_id.append(id[1])
    else:
        speaker_id.append(id)
    
    if indx[0:1] == "0":
        index.append(indx[1])
    else:
        index.append(indx)
    
    condition.append(con)
    
#Adding variables to dataframe
source["id"] = speaker_id
source["condition"] = condition
source["indx"] = index


source.file = os.path.join(sys.path[0], "Source/vox/") + source.file
source_files = source["file"].tolist()

features = surfboard.feature_extraction_multiprocessing.extract_features_from_paths(source_files, ["mfcc", 'f0_statistics', 'formants'], statistics_list=['mean'], sample_rate=44100, num_proc=3)
source = pd.concat([source, features], axis=1, sort=False)

source.head()

Extracting features from paths...: 100%|██████████| 400/400 [00:45<00:00,  8.89it/s]


,file,id,condition,indx,mfcc_mean_1,mfcc_mean_2,mfcc_mean_3,mfcc_mean_4,mfcc_mean_5,mfcc_mean_6,...,mfcc_mean_10,mfcc_mean_11,mfcc_mean_12,mfcc_mean_13,f0_mean,f0_std,f1,f2,f3,f4
0,/Users/jthiepler/Documents/GitHub/voice-classifier/Source/vox/08-2-09.wav,8,2,9,-521.157104,71.668144,2.469791,20.627808,4.826080,20.991871,...,1.341210,3.135353,5.152745,2.333008,133.692743,20.545576,491.477172,1342.991439,2512.375225,3654.489603
1,/Users/jthiepler/Documents/GitHub/voice-classifier/Source/vox/15-2-18.wav,15,2,18,-426.021210,82.447769,9.118480,26.285255,6.664119,17.659504,...,3.069383,-0.581232,0.661695,3.200798,140.788419,40.257182,1589.258816,2398.376728,3717.398821,4930.681489
2,/Users/jthiepler/Documents/GitHub/voice-classifier/Source/vox/10-2-06.wav,10,2,6,-486.532379,121.979752,-11.939642,27.319185,28.309999,18.014025,...,-7.956030,-2.497566,-0.142808,0.333498,104.349537,9.761097,1336.471135,2554.986935,3602.078416,4423.840882
3,/Users/jthiepler/Documents/GitHub/voice-classifier/Source/vox/11-2-12.wav,11,2,12,-508.592468,86.788513,14.815022,24.903181,15.770222,20.520506,...,4.253114,4.698157,2.170210,2.492243,113.496730,15.852238,306.285480,1180.233501,2478.401299,3531.644650
4,/Users/jthiepler/Documents/GitHub/voice-classifier/Source/vox/11-2-06.wav,11,2,6,-460.006317,93.882835,6.611367,23.654299,15.736868,18.022600,...,0.273451,-1.206936,-2.379301,-2.514076,149.818298,24.477652,2363.982891,3035.140787,3516.793534,4717.544723


In [4]:
#Create Training & Valuation Sets
def create_sets(subjects, n_train, n_val, conditions):
    train_set = pd.DataFrame()
    val_set = pd.DataFrame()
    selection = subjects
    
    if isinstance(conditions[0], tuple):
        for i in selection:
            tmp_train = source[(source.id==str(i)) & (source.condition==conditions[0][0])]
            tmp_train2 = source[(source.id==str(i)) & (source.condition==conditions[0][1])]
            tmp_train = tmp_train.sample(int(n_train/2))
            ind = tmp_train["indx"].tolist()
            tmp_train2 = tmp_train2[tmp_train2["indx"].isin(ind) == False]
            tmp_train2 = tmp_train2.sample(int(n_train/2))
            tmp_train = tmp_train.append(tmp_train2)
            train_set = train_set.append(tmp_train)
            ind = tmp_train["indx"].tolist()
                        
            tmp_val = source[(source.id==str(i)) & (source.condition==conditions[1])]
            tmp_val2 = tmp_val[tmp_val["indx"].isin(ind) == False]
            val_set = val_set.append(tmp_val2.sample(n_val))
    else:
        for i in selection:
            tmp_train = source[(source.id==str(i)) & (source.condition==conditions[0])]
            tmp_train = tmp_train.sample(n_train)
            train_set = train_set.append(tmp_train)
            ind = tmp_train["indx"].tolist()
                        
            tmp_val = source[(source.id==str(i)) & (source.condition==conditions[1])]
            tmp_val2 = tmp_val[tmp_val["indx"].isin(ind) == False]
            val_set = val_set.append(tmp_val2.sample(n_val))


    train_set = train_set.reset_index()
    val_set = val_set.reset_index()

    X_train = train_set.drop(train_set.columns[[0, 1, 2, 3, 4]], axis=1)
    X_val = val_set.drop(val_set.columns[[0, 1, 2, 3, 4]], axis=1)
    
    np.array(X_train)
    np.array(X_val)
    
    return train_set, val_set, X_train, X_val

In [5]:
#Pre-Processing
def processing(X_train, X_val, train_set, val_set):
    y_train = np.array(train_set["id"])
    y_val = np.array(val_set["id"])
    
    from sklearn.preprocessing import LabelEncoder
    from keras.utils.np_utils import to_categorical
    lb = LabelEncoder()
    y_train = to_categorical(lb.fit_transform(y_train))
    y_val = to_categorical(lb.fit_transform(y_val))
    
    from sklearn.preprocessing import StandardScaler
    ss = StandardScaler()
    X_train = ss.fit_transform(X_train)
    X_val = ss.transform(X_val)
    
    return y_train, y_val, X_train, X_val

In [10]:
#Build Model
def build_model(n_speakers):
    from keras.models import Sequential
    from keras.layers import Dense, Dropout, Activation, Flatten
    from keras.callbacks import EarlyStopping
    model = Sequential()
    model.add(Dense(19, input_shape=(19,), activation = 'relu'))
    #model.add(Dropout(0.1))
    model.add(Dense(128, activation = 'relu'))
    #model.add(Dropout(0.25))
    model.add(Dense(128, activation = 'relu'))
    #model.add(Dropout(0.5))
    model.add(Dense(10, activation = 'softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=1, mode='auto')
    
    return model, early_stop


In [11]:
#Train Model
def train_model(model, early_stop, log=0, graph=0):
    history = model.fit(X_train, y_train, batch_size=20, epochs=100,
                        validation_data=(X_val, y_val),
                        callbacks=[early_stop], verbose = log)
    
    train_accuracy = history.history['accuracy']
    val_accuracy = history.history['val_accuracy']
    
    if graph == 1:
        plt.figure(figsize=(12, 8))
        plt.plot(train_accuracy, label='Training Accuracy', color='#185fad')
        plt.plot(val_accuracy, label='Validation Accuracy', color='orange')
        plt.title('Training and Validation Accuracy by Epoch', fontsize = 25)
        plt.xlabel('Epoch', fontsize = 18)
        plt.ylabel('Categorical Crossentropy', fontsize = 18)
        plt.xticks(range(0,100,5), range(0,100,5))
        plt.legend(fontsize = 18);
    
    return val_accuracy

In [14]:
#Parameters
selection=(1,3,5,8,10,11,12,13,15,18)
n_train = 10
n_val = 20-n_train
conditions = [(("0","2"),"0"),("0","0"),("0","2"),("2","0"),("2","2")]
labels = ["NV+V-NV","NV-NV", "NV-V", "V-NV", "V-V"]
iterations = 10
n_speakers = len(selection)
log = 0
graph = 0

results = [ [] for _ in range(iterations) ]
avg_results= []
results_list = []
colours = ["orange", "blue", "green", "red", "purple"]

for i in range(len(conditions)):
    for j in range(iterations):
        
        #Creating sets
        [train_set, val_set, X_train, X_val] = create_sets(selection, n_train, n_val, conditions[i])
        
        #Pre-Processing
        [y_train, y_val, X_train, X_val] = processing(X_train, X_val, train_set, val_set)

        #Build model
        [model, early_stop] = build_model(n_speakers)

        #Train
        result = train_model(model, early_stop, log, graph)
        result = result[99]
        print(result)
        results[j].append(result)
        
        #Progress Counter
        print(str(1+j+i*iterations) + "/" + str(iterations * len(conditions)))
        
#Turn list into dataframe
results_df = pd.DataFrame(results, columns=labels)
results_df.head()

results_df.to_csv(r'/Users/jthiepler/Documents/GitHub/voice-classifier/results.csv')

#plt.figure(figsize=(12, 8))
#for i in range(len(conditions)):
    #plt.plot(avg_results[i], label=labels[i], color=colours[i])
#plt.title('Validation Accuracy by Epoch', fontsize = 25)
#plt.xlabel('Epoch', fontsize = 18)
#plt.ylabel('Categorical Crossentropy', fontsize = 18)
#plt.xticks(range(0,100,5), range(0,100,5))
#plt.legend(fontsize = 18);

0.8600000143051147
1/50
0.7799999713897705
2/50
0.8500000238418579
3/50
0.8299999833106995
4/50
0.7900000214576721
5/50
0.8199999928474426
6/50
0.8100000023841858
7/50
0.8700000047683716
8/50
0.75
9/50
0.8600000143051147
10/50
[[0.8600000143051147], [0.7799999713897705], [0.8500000238418579], [0.8299999833106995], [0.7900000214576721], [0.8199999928474426], [0.8100000023841858], [0.8700000047683716], [0.75], [0.8600000143051147]]
0.8600000143051147
11/50
0.8999999761581421
12/50
0.8899999856948853
13/50
0.8399999737739563
14/50
0.9100000262260437
15/50
0.9300000071525574
16/50
0.8899999856948853
17/50
0.9200000166893005
18/50
0.8899999856948853
19/50
0.9100000262260437
20/50
[[0.8600000143051147, 0.8600000143051147], [0.7799999713897705, 0.8999999761581421], [0.8500000238418579, 0.8899999856948853], [0.8299999833106995, 0.8399999737739563], [0.7900000214576721, 0.9100000262260437], [0.8199999928474426, 0.9300000071525574], [0.8100000023841858, 0.8899999856948853], [0.8700000047683716, 